In [1]:
from math import inf, copysign
X = START = HORIZONTAL = 0
Y = END = VERTICAL = 1
DIRECTION = DIAGONAL = 2

In [2]:
def print_data(data, num, min_nums, max_nums):
    """The size here represent the number of points it overlaps. That's why diagonals are 'short'"""
    num = min(num, len(data))
    print("First {} (from a total of {}) line segments:".format(num, len(data)))
    for segment in data[:num]:
        desc = "({}, {})\t->\t({}, {}):\t".format(segment[START][X], segment[START][Y], segment[END][X], segment[END][Y])
        if segment[DIRECTION] == HORIZONTAL:
            desc += "HORIZONTAL of size\t" + str(abs(segment[START][X] - segment[END][X]))
        elif segment[DIRECTION] == VERTICAL:
            desc += "VERTICAL of size\t" + str(abs(segment[START][Y] - segment[END][Y]))
        else:  # segment[DIRECTION] == DIAGONAL:
            desc += "DIAGONAL of size\t" + str(abs(segment[START][Y] - segment[END][Y]))
        print(desc)
    print("The grid goes from ({},{}) to ({},{})".format(min_nums[X], min_nums[Y], max_nums[X], max_nums[Y]))

In [3]:
def get_processed_input(input_path, data, verbose=True):
    """A line segment is repersented by two tuples o 2 integers (coordinates). First one is the start and the second one is the end
    Returns the max. and min. numbers found to make easier building of the grid"""
    with open(input_path, "rt") as f:
        max_nums = (-inf, -inf)
        min_nums = (inf, inf)
        raw_input = f.read()
        raw_lines = raw_input.split("\n")
        for line in raw_lines:
            coords_raw = line.split(" -> ")
            coords = [c.split(",") for c in coords_raw]
            # Segment format (3-sized tuple) -> (([START][X],[START][Y]), ([END][X],[END][Y]), DIRECTION)
            segment = [(int(coords[START][X]),int(coords[START][Y])), ((int(coords[END][X])), int(coords[END][Y])), DIAGONAL]
            if segment[START][X] == segment[END][X]:
                segment[DIRECTION] = VERTICAL
            elif segment[START][Y] == segment[END][Y]:
                segment[DIRECTION] = HORIZONTAL
                # If it's not a VERTICAL segment, neither an HORIZONTAL one, then it's a DIAGONAL. BUT,
                # if the angle it forms with the axis is not 45ª, then is discarded
            elif abs(segment[START][X] - segment[END][X]) == abs(segment[START][Y] - segment[END][Y]):
                segment[DIRECTION] = DIAGONAL
            else:
                if verbose:
                    print("({}, {})\t->\t({}, {}) discarded for being digonal and not form a 45ª with the axis".format(
                        segment[START][X], segment[START][Y], segment[END][X], segment[END][Y]))
            data.append(tuple(segment))
            max_nums = (max([max_nums[X], segment[START][X], segment[END][X]]), max([max_nums[Y], segment[START][Y], segment[END][Y]]))
            min_nums = (min([min_nums[X], segment[START][X], segment[END][X]]), min([min_nums[Y], segment[START][Y], segment[END][Y]]))
        if verbose:
            print_data(data, 10, min_nums, max_nums)
        return min_nums, max_nums

In [4]:
def draw_grid(grid, separator="", zero_char=".", min_grid_nums=(0,0), max_grid_nums=(inf,inf)):
    res = ""
    min_grid_nums = (min(min_grid_nums[X], 0), min(min_grid_nums[Y], 0))
    max_grid_nums = (min(max_grid_nums[X], len(grid)), min(max_grid_nums[Y], len(grid[0])))
    for y in range(min_grid_nums[Y], max_grid_nums[Y]):
        for x in range(min_grid_nums[X], max_grid_nums[X]):
            if grid[x][y]:
                res += str(grid[x][y])
            else:
                res += zero_char
            if x < len(grid[0]) - 1:
                res += separator
        if y < len(grid[0]) - 1:
            res += "\n"
    print(res)

In [5]:
def part_a(data, min_nums, max_nums, verbose=False):
    full_size = (max_nums[X] - min_nums[X], max_nums[Y] - min_nums[Y])
    grid = [[0 for i in range(full_size[X] + 1)] for j in range(full_size[Y] + 1)]
    if verbose:
        print("The size of the grid is {}x{} for a total of {} points" .format(full_size[X], full_size[Y], full_size[X] * full_size[Y]))
    offsets = (-min_nums[X], -min_nums[Y])  # If all the numbers are natural, these will be negative
    
    overlaped_points = set({})  # Probably the best way of avoiding duplicates
    max_overlaps = 0      # debug/curiosity
    max_overlaped = None  # debug/curiosity
    for segment in data:
        if segment[DIRECTION] == DIAGONAL:
            continue
        seg_mins = (min(segment[START][X], segment[END][X]), min(segment[START][Y], segment[END][Y]))
        seg_maxs = (max(segment[START][X], segment[END][X]), max(segment[START][Y], segment[END][Y]))
        for x in range(seg_mins[X], seg_maxs[X] + 1):
            for y in range(seg_mins[Y], seg_maxs[Y] + 1):
                grid_x = x+offsets[X]
                grid_y = y+offsets[Y]
                grid[grid_x][grid_y] += 1
                overlaps = grid[grid_x][grid_y]
                point = (x, y)
                if overlaps > 1:
                    overlaped_points.add(point)
                if overlaps > max_overlaps:
                    max_overlaps = overlaps
                    max_overlaped = point
    if verbose:
        draw_grid(grid, separator="  ")
    return len(overlaped_points), max_overlaped, max_overlaps

In [6]:
def part_b(data, min_nums, max_nums, diagonals=False, verbose=False):
    full_size = (max_nums[X] - min_nums[X], max_nums[Y] - min_nums[Y])
    grid = [[0 for i in range(full_size[X] + 2)] for j in range(full_size[Y] + 1)]
    if verbose:
        print("The size of the grid is {}x{} for a total of {} points" .format(full_size[X], full_size[Y], full_size[X] * full_size[Y]))
    offsets = (-min_nums[X], -min_nums[Y])
    
    overlaped_points = set({})  # Probably the best way of avoiding duplicates
    max_overlaps = 0      # debug/curiosity
    max_overlaped = None  # debug/curiosity
    for segment in data:
        distances = (segment[END][X] - segment[START][X] , segment[END][Y] - segment[START][Y])
        directions = [0,0]
        if distances[X]:
            directions[X] = copysign(1, distances[X])
        if distances[Y]:
            directions[Y] = copysign(1, distances[Y])
        directions = tuple([int(diresction) for diresction in directions])  # copysing returns a float
        for i in range(max([abs(d) for d in distances]) + 1):
            x = segment[START][X] + i * directions[X]
            y = segment[START][Y] + i * directions[Y]
            grid_x = x + offsets[X]
            grid_y = y + offsets[Y]
            try:
                grid[grid_x][grid_y] += 1
                overlaps = grid[grid_x][grid_y]
                point = (x, y)
                if overlaps > 1:
                    overlaped_points.add(point)
                if overlaps > max_overlaps:
                    max_overlaps = overlaps
                    max_overlaped = point
            except:
                print(x + offsets[X], y + offsets[Y])
    if verbose:
        draw_grid(grid, separator="  ")
    return len(overlaped_points), max_overlaped, max_overlaps

In [7]:
data = []
input_path = "input.txt"
min_nums, max_nums = get_processed_input(input_path, data)

First 10 (from a total of 500) line segments:
(510, 771)	->	(510, 322):	VERTICAL of size	449
(753, 99)	->	(753, 280):	VERTICAL of size	181
(160, 330)	->	(33, 330):	HORIZONTAL of size	127
(700, 793)	->	(700, 892):	VERTICAL of size	99
(327, 168)	->	(327, 690):	VERTICAL of size	522
(264, 203)	->	(264, 839):	VERTICAL of size	636
(135, 134)	->	(314, 134):	HORIZONTAL of size	179
(209, 759)	->	(41, 759):	HORIZONTAL of size	168
(474, 514)	->	(491, 531):	DIAGONAL of size	17
(977, 988)	->	(42, 53):	DIAGONAL of size	935
The grid goes from (11,10) to (988,988)


In [8]:
# There might be more points with the max. overlaps, but only the first one that reaches that value is shown
sol_a, max_overlaped_a, max_overlaps_a = part_a(data, min_nums, max_nums, verbose=False)
print("SOL A: [{}] points have at least two lines overlapping them, being ({}, {}) \
the most overlapped one with a total of {} lines".format(sol_a, max_overlaped_a[0], max_overlaped_a[1], max_overlaps_a))

SOL A: [6397] points have at least two lines overlapping them, being (773, 731) the most overlapped one with a total of 3 lines


In [9]:
sol_b, max_overlaped_b, max_overlaps_b = part_b(data, min_nums, max_nums, diagonals=True, verbose=False)
print("SOL B: [{}] points have at least two lines overlapping them, being ({}, {}) \
the most overlapped one with a total of {} lines".format(sol_b, max_overlaped_b[0], max_overlaped_b[1], max_overlaps_b))

SOL B: [22335] points have at least two lines overlapping them, being (510, 516) the most overlapped one with a total of 6 lines


## TEST

### A

Sol = 5

### B

Sol = 12

In [10]:
data_test = [
    ((0,9),(5,9),HORIZONTAL),
    ((8,0),(0,8),DIAGONAL),
    ((9,4),(3,4),HORIZONTAL),
    ((2,2),(2,1),VERTICAL),
    ((7,0),(7,4),VERTICAL),
    ((6,4),(2,0),DIAGONAL),
    ((0,9),(2,9),HORIZONTAL),
    ((3,4),(1,4),HORIZONTAL),
    ((0,0),(8,8),DIAGONAL),
    ((5,5),(8,2),DIAGONAL)
]
min_nums_test = (0,0)
max_nums_test = (9,9)
sol_test_a, max_overlaped_test_a, max_overlaps_test_a = part_a(data_test, min_nums_test, max_nums_test, verbose=True)
print("SOL Test A: [{}] points have at least two lines overlapping them, being ({}, {}) \
the most overlapped one with a total of {} lines".format(sol_test_a, max_overlaped_test_a[0], max_overlaped_test_a[1],
                                                         max_overlaps_test_a))

The size of the grid is 9x9 for a total of 81 points
.  .  .  .  .  .  .  1  .  .
.  .  1  .  .  .  .  1  .  .
.  .  1  .  .  .  .  1  .  .
.  .  .  .  .  .  .  1  .  .
.  1  1  2  1  1  1  2  1  1
.  .  .  .  .  .  .  .  .  .
.  .  .  .  .  .  .  .  .  .
.  .  .  .  .  .  .  .  .  .
.  .  .  .  .  .  .  .  .  .
2  2  2  1  1  1  .  .  .  .
SOL Test A: [5] points have at least two lines overlapping them, being (7, 4) the most overlapped one with a total of 2 lines


In [11]:
sol_test_b, max_overlaped_test_b, max_overlaps_test_b = part_b(data_test, min_nums_test, max_nums_test, verbose=True)
print("SOL Test B: [{}] points have at least two lines overlapping them, being ({}, {}) \
the most overlapped one with a total of {} lines".format(sol_test_b, max_overlaped_test_b[0], max_overlaped_test_b[1],
                                                         max_overlaps_test_b))

The size of the grid is 9x9 for a total of 81 points
1  .  1  .  .  .  .  1  1  .  
.  1  1  1  .  .  .  2  .  .  
.  .  2  .  1  .  1  1  1  .  
.  .  .  1  .  2  .  2  .  .  
.  1  1  2  3  1  3  2  1  1  
.  .  .  1  .  2  .  .  .  .  
.  .  1  .  .  .  1  .  .  .  
.  1  .  .  .  .  .  1  .  .  
1  .  .  .  .  .  .  .  1  .  
2  2  2  1  1  1  .  .  .  .  
.  .  .  .  .  .  .  .  .  .  
SOL Test B: [12] points have at least two lines overlapping them, being (4, 4) the most overlapped one with a total of 3 lines
